We are representing all words as vectors for keywords and categories. We summarizing it and after that we are calculating simularity between then and seraching fo k-nearest neighbors. For tests we are using NearestNeighbors bruteforce algorithm, it should be changed to Local Sencitivity Hashing NearestNeighbors clustering.

Lets define KeywordsClassifier class:

In [1]:
import numpy
from sklearn.neighbors import NearestNeighbors
import spacy


class KeywordsClassifier(object):

    def __init__(self, nearestNeighborsLearner, categories, topN):
        self.nearestNeighborsLearner = nearestNeighborsLearner 
        self.categories = categories 
        self.topN = topN 

    @classmethod
    def CreateKeywordsClassifier(cls, nlp, categories, topNCount):
        '''
        @param nlp: spacy dictinary 
        @param categories: list of categories
        @param topNCount: number of categories to chose
        @return: instance of KeywordsClassifier class
        '''
        catDic = KeywordsClassifier.GetVerticalsVectorsDict(nlp, categories)
        catArray = numpy.array(list(catDic.values()))
        nn = NearestNeighbors(topNCount, algorithm='brute', metric='cosine') #for production we should use LSHForest instead
        nn.fit(catArray)
        return KeywordsClassifier(nn, list(catDic.keys()), topNCount)

    @staticmethod
    def GetVerticalsVectorsDict(nlp, verticals):
        verticalsDict = {}

        for category in verticals:
            text = nlp(category.replace("&", ""))
            verticalsDict[category] = text.vector

        return verticalsDict


    def ClassifyKeywords(self, keywordsVector):
        '''
        @param keywordsVector: resulting keywords vector
        @return: ordered list of tuples representing of top N categories names with cosine similarities
        '''
        result = []
        distances, indices = self.nearestNeighborsLearner.kneighbors(keywordsVector.reshape(1, -1))
        curIter = 0

        for curIndex in numpy.nditer(indices):
            category = self.categories[curIndex]
            cosDistance = distances[0][curIter]
            tuple = (category, 1 - cosDistance)
            result.append(tuple)
            curIter += 1

        return result



# categories = { 'fruits', 'cars', 'bikes', 'animals'}
# keywords = 'BMW NISSAN TOYOTA'
# topN = 1
# classifier = KeywordsClassifier.CreateKeywordsClassifier( nlp, verticals, topN)
# result = classifier.ClassifyKeywords(keywordsVect)
# print(result)

hello world


Before we can start we need to load the spacy dictinary:

In [ ]:
if 'nlp' not in globals():
    print("Loading dictinary...")
    nlp = spacy.load('en')
    print("Dictinary loaded.")